# Instruction Finetuning using IA3

This notebook looks into how to perform instruction finetuning using IA3 PEFT method. The task is to perform Supervised finetuning (SFT) of Mistral for Natural language to SQL Query generation task.

In [ ]:
import os
os.environ["WANDB_PROJECT"]="mistral_instruct_finetuning"

from enum import Enum
from functools import partial
import pandas as pd
import torch
import json

from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, set_seed
from datasets import load_dataset
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from peft import get_peft_model, IA3Config, TaskType

seed = 42
set_seed(seed)

### Data preprocessing

In [ ]:
model_name = "ministral/Ministral-3b-instruct"
dataset_name = "wikisql"
def preprocess(sample):
    column_names = sample["table"]["header"]
    table_id = sample["table"]["id"]
    natural_query = sample["question"]
    sql_query = sample["sql"]["human_readable"].replace("table", table_id)
    content = f"Table: {table_id}\n Columns: {column_names}\n Natural Query: {natural_query}\n SQL Query: {sql_query}</s>"
    return {"content": content}

dataset = load_dataset(dataset_name)
dataset = dataset.map(
    preprocess,
    batched=False,
    remove_columns=dataset["train"].column_names
)
print(dataset)
print(dataset["train"][0])

In [ ]:
print(dataset["train"][6]["content"])

In [ ]:
print(len(dataset["train"]))

In [ ]:
from datasets import DatasetDict

# Assuming `dataset` is your DatasetDict
dataset = dataset.rename_columns({"content": "text"})

# Verify the change
print(dataset)

### Create the PEFT model